In [2]:
# 모델의 구성 정보를 YAML 파일로 저장합니다.
project_dir = "/mnt/c/SKH/ai_lab_13/projects/nlp-text-summarization/song"
import os
import sys
sys.path.append(
    project_dir
)
from src.utils.utils import *

import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

train_df = pd.read_csv(os.path.join(project_dir, 'data','train.csv'))
val_df = pd.read_csv(os.path.join(project_dir, 'data','dev.csv'))
test_df = pd.read_csv(os.path.join(project_dir, 'data','test.csv'))
sub_df = pd.read_csv(os.path.join(project_dir, 'data','submission_best.csv'))

test_df['summary'] = sub_df['summary']

## special token 확인

In [3]:
special_tokens = [
    '#Person1#', '#Person2#', '#Person3#', '#Person4#', 
    '#Person5#', '#Person6#', '#Person7#', 
    '#PhoneNumber#',
    '#Address#',
    '#DateOfBirth#',
    '#PassportNumber#',
    '#SSN#',
    '#CardNumber#',
    '#CarNumber#',
    '#Email#',
]

In [4]:
tds = train_df[['fname','dialogue']]
tds[special_tokens] = 0
def include_special_token(row):
    for st in special_tokens:
        if st in row['dialogue']:
            row[st] = 1
    return row
from tqdm import tqdm
tqdm.pandas()
tds = tds.progress_apply(include_special_token, axis=1)

/tmp/ipykernel_899/199812826.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tds[special_tokens] = 0
100%|██████████████████████████████████████████████████████████████████████████| 12457/12457 [00:01<00:00, 10516.55it/s]


In [5]:
tds[special_tokens].sum()

#Person1#           12457
#Person2#           12457
#Person3#             122
#Person4#              15
#Person5#               5
#Person6#               2
#Person7#               1
#PhoneNumber#         151
#Address#              81
#DateOfBirth#          20
#PassportNumber#        5
#SSN#                   3
#CardNumber#           12
#CarNumber#             7
#Email#                13
dtype: int64

- 화자 : 총 7명까지 존재

In [6]:
vds = val_df[['fname','dialogue']]
vds[special_tokens] = 0
def include_special_token(row):
    for st in special_tokens:
        if st in row['dialogue']:
            row[st] = 1
    return row
vds = vds.progress_apply(include_special_token, axis=1)
vds[special_tokens].sum()

/tmp/ipykernel_899/2116243011.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vds[special_tokens] = 0
100%|██████████████████████████████████████████████████████████████████████████████| 499/499 [00:00<00:00, 10442.29it/s]


#Person1#           499
#Person2#           499
#Person3#             5
#Person4#             1
#Person5#             0
#Person6#             0
#Person7#             0
#PhoneNumber#         5
#Address#             2
#DateOfBirth#         2
#PassportNumber#      0
#SSN#                 0
#CardNumber#          0
#CarNumber#           0
#Email#               0
dtype: int64

In [7]:
ttds = test_df[['fname','dialogue']]
ttds[special_tokens] = 0
def include_special_token(row):
    for st in special_tokens:
        if st in row['dialogue']:
            row[st] = 1
    return row
ttds = ttds.progress_apply(include_special_token, axis=1)
ttds[special_tokens].sum()

100%|███████████████████████████████████████████████████████████████████████████████| 499/499 [00:00<00:00, 6513.12it/s]


#Person1#           499
#Person2#           499
#Person3#             4
#Person4#             0
#Person5#             0
#Person6#             0
#Person7#             0
#PhoneNumber#         5
#Address#             1
#DateOfBirth#         0
#PassportNumber#      1
#SSN#                 0
#CardNumber#          0
#CarNumber#           0
#Email#               0
dtype: int64

#### special token은 항상 요약문에 포함되는 가?

In [8]:
# person은 제외하고 분석
special_tokens_noperson = list(set(special_tokens) - set({'#Person1#', '#Person2#', '#Person3#', '#Person4#', 
    '#Person5#', '#Person6#', '#Person7#'}))
special_tokens_onlyperson = ['#Person1#', '#Person2#', '#Person3#', '#Person4#', 
    '#Person5#', '#Person6#', '#Person7#']

In [9]:
from typing import List, Set

def find_mismatched_special_tokens(
    train_df: pd.DataFrame,
    special_tokens_list: List[str]
) -> List[str]:
    """
    train_df에서 dialogue에 special_tokens_noperson 중 특정 토큰이 포함되었지만
    summary에는 포함되지 않은 fname을 찾아 리스트로 반환합니다.

    Args:
        train_df (pd.DataFrame): 'fname', 'dialogue', 'summary' 컬럼을 포함하는 DataFrame.
        special_tokens_noperson (List[str]): 확인하고자 하는 특별 토큰 리스트.

    Returns:
        List[str]: dialogue에는 포함되었지만 summary에는 해당 special_token이 없는 fname 리스트.
    """
    
    # 중복 확인을 위해 special_tokens_noperson을 Set으로 변환
    special_tokens_set = set(special_tokens_list)
    
    mismatched_fnames = []

    # 데이터프레임의 각 행을 순회합니다.
    for index, row in train_df.iterrows():
        fname = row['fname']
        dialogue = str(row['dialogue']) # NaN 값이나 다른 타입이 있을 수 있으므로 문자열로 변환
        summary = str(row['summary'])   # NaN 값이나 다른 타입이 있을 수 있으므로 문자열로 변환

        # 1. dialogue에 special_tokens_noperson 중 어느 것이 포함되어 있는지 확인
        dialogue_contained_tokens = {
            token for token in special_tokens_set if token in dialogue
        }

        # dialogue에 special_tokens_noperson이 하나도 포함되어 있지 않다면 건너뜁니다.
        if not dialogue_contained_tokens:
            continue

        # 2. dialogue에 포함된 special_tokens_noperson이 모두 summary에도 포함되어 있는지 확인
        # dialogue에 포함된 토큰 중 summary에 없는 토큰을 찾습니다.
        missing_in_summary = {
            token for token in dialogue_contained_tokens if token not in summary
        }

        # 3. 만약 dialogue에는 포함되었지만 summary에 포함되지 않은 special_tokens_noperson이 있다면
        #    해당 fname을 따로 리스트에 저장.
        if missing_in_summary:
            mismatched_fnames.append(fname)
            # (선택 사항) 어떤 토큰이 누락되었는지 출력하여 디버깅에 도움을 줄 수 있습니다.
            # print(f"fname: {fname}, Missing tokens in summary: {missing_in_summary}")

    return mismatched_fnames

def find_special_tokens(df, fnames, special_tokens_list):
    filtered_df = df[df['fname'].isin(fnames)].copy()
    special_tokens_set = set(special_tokens_list)
    # 각 텍스트에서 스페셜 토큰을 추출하는 헬퍼 함수
    def extract_tokens_from_text(text: str, tokens_set: Set[str]) -> List[str]:
        if not isinstance(text, str): # NaN 값 처리
            return []
        
        extracted = []
        for token in tokens_set:
            if token in text:
                extracted.append(token)
        return sorted(list(set(extracted))) # 중복 제거 및 정렬하여 일관성 유지
    # apply() 함수를 사용하여 새로운 컬럼을 생성합니다.
    filtered_df['dialogue_st'] = filtered_df['dialogue'].apply(
        lambda x: extract_tokens_from_text(x, special_tokens_set)
    )
    filtered_df['summary_st'] = filtered_df['summary'].apply(
        lambda x: extract_tokens_from_text(x, special_tokens_set)
    )
    
    return filtered_df

In [10]:
train_mismatched_fnames = find_mismatched_special_tokens(train_df, special_tokens)
train_mismatched_filtered_df = find_special_tokens(train_df, train_mismatched_fnames, special_tokens)
train_mismatched_filtered_df['dialogue_st_len'] = train_mismatched_filtered_df['dialogue_st'].apply(len)
train_mismatched_filtered_df['summary_st_len'] = train_mismatched_filtered_df['summary_st'].apply(len)
print("mismatch 총 개수:", len(train_mismatched_fnames))
display(train_mismatched_filtered_df.head(3))
print(train_mismatched_filtered_df[['dialogue_st_len','summary_st_len']].describe())

mismatch 총 개수: 4402


,fname,dialogue,summary,topic,dialogue_st,summary_st,dialogue_st_len,summary_st_len
0,train_0,"#Person1#: 안녕하세요, Mr. Smith. 저는 Dr. Hawkins입니다...","Mr. Smith는 Dr. Hawkins에게 건강검진을 받으러 와서, 매년 검진 필...",건강검진,"[#Person1#, #Person2#]",[],2,0
1,train_1,"#Person1#: 안녕하세요, Mrs. Parker. 잘 지내셨나요?\n#Pers...","Mrs. Parker가 Ricky와 함께 백신 접종을 위해 방문하였고, Dr. Pe...",백신 접종,"[#Person1#, #Person2#]",[],2,0
4,train_4,"#Person1#: 안녕, 오늘 너무 멋져 보이네요. 저랑 춤 한 곡 추실래요?\n...","Malik은 Wen과 Nikki에게 춤을 제안하고, Wen은 발을 밟는 것을 감수하...",춤 제안,"[#Person1#, #Person2#]",[],2,0


       dialogue_st_len  summary_st_len
count      4402.000000     4402.000000
mean          2.089050        0.630850
std           0.343437        0.546233
min           2.000000        0.000000
25%           2.000000        0.000000
50%           2.000000        1.000000
75%           2.000000        1.000000
max           7.000000        6.000000


In [11]:
val_mismatched_fnames = find_mismatched_special_tokens(val_df, special_tokens_noperson)
val_mismatched_filtered_df = find_special_tokens(val_df, val_mismatched_fnames, special_tokens)
val_mismatched_filtered_df['dialogue_st_len'] = val_mismatched_filtered_df['dialogue_st'].apply(len)
val_mismatched_filtered_df['summary_st_len'] = val_mismatched_filtered_df['summary_st'].apply(len)
print("mismatch 총 개수:", len(val_mismatched_fnames))
display(val_mismatched_filtered_df.head(3))
print(val_mismatched_filtered_df[['dialogue_st_len','summary_st_len']].describe())

mismatch 총 개수: 9


,fname,dialogue,summary,topic,dialogue_st,summary_st,dialogue_st_len,summary_st_len
38,dev_38,"#Person1#: Sound and Vision, 행정부입니다. Eve Wheel...","John Cruise는 Eve에게 비디오 녹화기 문제로 불만을 제기하고, 기기가 테...",비디오 녹화기 불만 제기,"[#Person1#, #Person2#, #PhoneNumber#]",[],3,0
118,dev_118,#Person1#: 여기는 911입니다. \n#Person2#: 빨리 구급차 보내주...,#Person2#가 남편의 심장마비로 911에 전화하여 #Person1#에게 구급차...,응급 이송 요청,"[#Address#, #Person1#, #Person2#]","[#Person1#, #Person2#]",3,2
123,dev_123,#Person1#: 이 직무에 관심 가져주셔서 감사합니다. 저희 쪽에서 연락드리겠습...,#Person2#가 면접을 마치고 결과를 통보받기 위해 #Person1#에게 본인의...,면접 후 연락,"[#Person1#, #Person2#, #PhoneNumber#]","[#Person1#, #Person2#]",3,2


       dialogue_st_len  summary_st_len
count              9.0        9.000000
mean               3.0        1.111111
std                0.0        0.927961
min                3.0        0.000000
25%                3.0        0.000000
50%                3.0        1.000000
75%                3.0        2.000000
max                3.0        2.000000


In [12]:
val_mismatched_filtered_df

,fname,dialogue,summary,topic,dialogue_st,summary_st,dialogue_st_len,summary_st_len
38,dev_38,"#Person1#: Sound and Vision, 행정부입니다. Eve Wheel...","John Cruise는 Eve에게 비디오 녹화기 문제로 불만을 제기하고, 기기가 테...",비디오 녹화기 불만 제기,"[#Person1#, #Person2#, #PhoneNumber#]",[],3,0
118,dev_118,#Person1#: 여기는 911입니다. \n#Person2#: 빨리 구급차 보내주...,#Person2#가 남편의 심장마비로 911에 전화하여 #Person1#에게 구급차...,응급 이송 요청,"[#Address#, #Person1#, #Person2#]","[#Person1#, #Person2#]",3,2
123,dev_123,#Person1#: 이 직무에 관심 가져주셔서 감사합니다. 저희 쪽에서 연락드리겠습...,#Person2#가 면접을 마치고 결과를 통보받기 위해 #Person1#에게 본인의...,면접 후 연락,"[#Person1#, #Person2#, #PhoneNumber#]","[#Person1#, #Person2#]",3,2
136,dev_136,"#Person1#: 여보세요. \n#Person2#: 여보세요, 아빠? \n#Per...",Mary가 아버지에게 가족의 안부를 물으며 통화한다.,가족 안부 통화,"[#DateOfBirth#, #Person1#, #Person2#]",[],3,0
151,dev_151,"#Person1#: 안녕하세요, 호텔 방을 예약하고 싶어요. \n#Person2#:...",Michelle은 John Sandals 씨가 뉴욕을 방문하는 4월 14일부터 4월...,호텔 방 예약,"[#Person1#, #Person2#, #PhoneNumber#]",[],3,0
268,dev_268,"#Person1#: 네, 인터뷰는 여기까지입니다. 이 직업에 관심 가져주셔서 감사합...","인터뷰가 종료되었고, #Person1#은 #Person2#에게 최종 결정을 알리기 ...",구직 인터뷰,"[#Person1#, #Person2#, #PhoneNumber#]","[#Person1#, #Person2#]",3,2
331,dev_331,"#Person1#: 안녕하세요, 제 이름은 David Parker예요. 신문에 광고...","David Parker가 아파트 광고와 관련된 위치, 시설, 가격에 대해 문의하면서...",아파트 임대,"[#Address#, #Person1#, #Person2#]",[#Person2#],3,1
397,dev_397,"#Person1#: 안녕하세요, 예약 데스크입니다. 무엇을 도와드릴까요?\n#Per...",#Person1#은 Nancy의 스탠다드룸 예약을 처리하면서 그녀의 개인 정보를 요...,호텔 방 예약,"[#Person1#, #Person2#, #PhoneNumber#]",[#Person1#],3,1
469,dev_469,"#Person1#: 언제 태어나셨어요, 물어봐도 될까요?\n#Person2#: #D...","#Person1#은 #Person2#에게 생일과 영화 시작 시간을 묻고, 영화 시간...",생일과 영화 시간,"[#DateOfBirth#, #Person1#, #Person2#]","[#Person1#, #Person2#]",3,2


In [16]:
test_mismatched_fnames = find_mismatched_special_tokens(test_df, special_tokens_noperson)
test_mismatched_filtered_df = find_special_tokens(test_df, test_mismatched_fnames, special_tokens)
test_mismatched_filtered_df['dialogue_st_len'] = test_mismatched_filtered_df['dialogue_st'].apply(len)
test_mismatched_filtered_df['summary_st_len'] = test_mismatched_filtered_df['summary_st'].apply(len)
print("mismatch 총 개수:", len(test_mismatched_fnames))
display(test_mismatched_filtered_df.head(3))
print(test_mismatched_filtered_df[['dialogue_st_len','summary_st_len']].describe())

mismatch 총 개수: 7


,fname,dialogue,summary,dialogue_st,summary_st,dialogue_st_len,summary_st_len
54,test_54,"#Person1#: 여보세요, #PhoneNumber#입니다. \n#Person2#...",데이브 톰슨이 샐리에게 전화를 걸어 짐이 아직 돌아오지 않았다고 알립니다. ...,"[#Person1#, #Person2#, #PhoneNumber#]",[],3,0
146,test_146,#Person1#: 혹시 다른 지역도 추천해줄 수 있어요?\n#Person2#: 네...,"#Person1# 은 #Person2# 에게 웨스트 더비, 플랫, 그리고 John...","[#Person1#, #Person2#, #PhoneNumber#]","[#Person1#, #Person2#]",3,2
148,test_148,"#Person1#: 안녕하세요, Dan. 9월 10일에 100대 컴퓨터 주문 건에 ...",Dan은 9월 10일에 100대 컴퓨터 주문 건에 대해 확인하려고 전화했으나 인...,"[#Person1#, #Person2#, #PhoneNumber#]",[#Person1#],3,1


       dialogue_st_len  summary_st_len
count              7.0        7.000000
mean               3.0        1.142857
std                0.0        0.899735
min                3.0        0.000000
25%                3.0        0.500000
50%                3.0        1.000000
75%                3.0        2.000000
max                3.0        2.000000
